Decision Tree Regression

In [1]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import GridSearchCV
# 其他库已在 sample.ipynb导入了 例如：
# import numpy as np
# from sklearn import metrics
# import matplotlib.pyplot as plt

3.1 基准（未剪枝）决策树模型
首先，我们训练一个使用默认参数的决策树。未剪枝的树会尽可能生长，导致在训练集上表现完美，但在测试集上表现不佳（即过度拟合）。

In [ ]:
# 1. 初始化模型
# 使用 random_state=0 确保结果可复现
dt_baseline = DecisionTreeRegressor(random_state=0)

# 2. 训练模型
dt_baseline.fit(X_train, y_train)

# 3. 评估模型
y_pred_dt_baseline = dt_baseline.predict(X_test)

print("---The baseline decision tree model (unpruned)---")
print(f'R2 Score: {metrics.r2_score(y_test, y_pred_dt_baseline):.4f}')
print(f'MAE: {metrics.mean_absolute_error(y_test, y_pred_dt_baseline):.4f}')
# sample.ipynb 中使用了 RMSE
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred_dt_baseline)):.4f}')

# 检查树的深度
print(f'\nMaximum depth of decision tree: {dt_baseline.get_depth()}')

3.2 使用 GridSearchCV 优化决策树（剪枝）
基准模型很可能过度拟合（测试集 R² 较低，RMSE 较高）。现在我们使用 GridSearchCV来搜索最佳的“剪枝”超参数，以提高模型的泛化能力。

我们将调整 max_depth（最大深度）和 min_samples_leaf（叶节点最小样本数）。

In [ ]:
# 1. 定义参数网格
param_grid = {
    'max_depth': [3, 5, 7, 9, 11],          # Try different tree depths
    'min_samples_leaf': [5, 10, 20, 30],      # Minimum number of samples required for leaf nodes
    'min_samples_split': [10, 20, 40]     # Minimum number of samples required for internal node splitting
}

# 2. 初始化 GridSearchCV
# 我们使用 'r2' 作为评分标准
# cv=5 表示5折交叉验证
grid_search = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=0),
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1, # 使用所有可用的CPU
    verbose=1  # 显示搜索过程
)

# 3. 在训练数据上运行网格搜索
print("Start searching for optimal parameters...")
grid_search.fit(X_train, y_train)

# 4. 打印最佳参数
print("\nSearch complete.")
print("Find the optimal parameter combination：")
print(grid_search.best_params_)

# 5. 获取最佳模型
# grid_search.best_estimator_ 是使用最佳参数在 *整个* X_train 上自动重新训练好的模型
dt_optimized = grid_search.best_estimator_

3.3 评估优化后的决策树
现在我们使用 grid_search 找到的最佳模型（dt_optimized）来评估其在测试集上的真实表现。


In [ ]:
# 1. 使用优化后的模型进行预测
y_pred_dt_optimized = dt_optimized.predict(X_test)

print("--- 3.3 Evaluation of the optimized decision tree (after pruning) ---")
print(f'R2 Score: {metrics.r2_score(y_test, y_pred_dt_optimized):.4f}')
print(f'MAE: {metrics.mean_absolute_error(y_test, y_pred_dt_optimized):.4f}')
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred_dt_optimized)):.4f}')

print(f'\nMaximum depth of the optimized tree： {dt_optimized.get_depth()}')

3.4 商业洞察：特征重要性 (Feature Importance)
决策树模型可以告诉我们哪些特征对预测“保险费用”最重要。这对于小组作业要求的商业解读至关重要。

In [ ]:
# 1. 获取特征重要性
importances = dt_optimized.feature_importances_

# 2. 获取特征名称
# 确保 X_train 在此单元格中是 DataFrame 格式
feature_names = X_train.columns

# 3. 创建一个 DataFrame 便于查看
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 4. 打印重要的特征
print("--- Feature Importance Ranking ---")
print(importance_df)

# 5. 可视化特征重要性
plt.figure(figsize=(10, 6))
plt.title('Optimized Decision Tree - Feature Importance')
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.gca().invert_yaxis() # 将最重要的特征放在顶部
plt.show()

3.5 商业洞察：可视化决策树
为了更好地理解模型是如何做出决策的，我们可以（部分）可视化这棵优化后的树。参考 T5 Decision Tree 的方法，我们只绘制顶部几层（例如 max_depth=3），以保持可读性。

In [ ]:
plt.figure(figsize=(20, 12)) # 调大图像尺寸以便看清

# 使用 plot_tree 可视化
# 我们只显示前3层 (max_depth=3)，以保持清晰
plot_tree(
    dt_optimized,
    feature_names=feature_names,
    filled=True,           # 用颜色填充节点
    rounded=True,          # 圆角矩形框
    fontsize=10,           # 字体大小
    max_depth=3            # !! 仅用于可视化的深度
)

plt.title(f'Optimized Decision Tree (Visualized up to 3 levels, actual depth: {dt_optimized.get_depth()})')
plt.show()